In [2]:
from get_datasets import get_data
train_dataset, val_dataset, test_dataset, vocab = get_data('SentimentClassifier/')

In [3]:
from torch.utils.data import DataLoader

def sorted_collate(batch):
    x, y = zip(*batch)
    order = np.argsort([len(xi) for xi in x])[::-1]
    x = [torch.LongTensor(x[i]) for i in order]
    y = torch.stack([y[i] for i in order])[:, None]
    return x,y

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, collate_fn=sorted_collate)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False, collate_fn=sorted_collate)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False, collate_fn=sorted_collate)

In [4]:
from torch import jit
import torch
from torch import nn

class SkipGRUCell(jit.ScriptModule):
    def __init__(self, input_size, hidden_size):
        """Simple SkipGRU cell.
        Args:
            input_size (int): size of input space of GRU
            hidden_size (int): size of hidden space of GRU
        """
        super().__init__()
        self.inner_gru = nn.GRUCell(input_size, hidden_size)

    @jit.script_method
    def forward(self, input, hidden, mix):
        """
        Args:
            input (torch.FloatTensor): input vector for the current step
            hidden (torch.FloatTensor): hidden vector for the previous step
            mix (torch.FloatTensor): mixture vector which defines proportion of mix between previous step and current.
        Returns:
            hidden (torch.FloatTensor): hidden vetor for the current step
        """
        # type: (Tensor, Tensor, Tensor) -> Tensor

        output = self.inner_gru(input, hidden)
        output = output * mix + hidden * (1 - mix)

        return output

from typing import List, Tuple

class SkipGRULayer(jit.ScriptModule):
    __constants__ = ['hidden_size']
    def __init__(self, input_size, hidden_size):
        """
        Layer consisted of SkipGRU cells.
        Args:
            input_size (int): size of input space of GRU
            hidden_size (int): size of hidden space of GRU
        """
        super().__init__()
        self.cell = SkipGRUCell(input_size, hidden_size)
        self.hidden_size = hidden_size

    @jit.script_method
    def forward(self, input, mix):
        """
        Forward pass on SkipGRU layer.
        Args:
            input (torch.FloatTensor): inputs for the SkipGRU layer of form [BxTxN]
            mix (torch.FloatTensor): mixture coefficients for thehidden steps of form [BxTx1]
        Returns:
            h (torch.FloatTensor): history of outputs by steps. Size [BxTxH]
            o (torch.FloatTensor): last hidden step of size [BxH]
        """
        # type: (Tensor, Tensor) -> Tuple[Tensor, Tensor]
        hidden = torch.zeros(input.shape[0], self.hidden_size,
                             device=self.cell.inner_gru.weight_ih.device)
        inputs = input.unbind(1)
        mixtures = mix.unbind(1)
        outputs = jit.annotate(List[Tensor], [])
        for i in range(len(inputs)):
            hidden = self.cell(inputs[i], hidden, mixtures[i])
            outputs += [hidden]
        return torch.stack(outputs, 1), hidden


class SkipGRU(nn.Module):
    def __init__(self, input_size=None, hidden_size=None, layer=None, do_copy_weights=False):
        """
        SkipRNN layer with trivial binarizer (All betas are directly parametrized.)
        Args:
            input_size (int): size of input space for SkipGRU
            hidden_size (int): size of hidden space for SkipGRU
            layer (nn.GRU): original GRU layers to copy sizes and make a link to weights
        """
        super().__init__()
        if layer is not None:
            self.layer = SkipGRULayer(layer.input_size, layer.hidden_size)
            self.impute_weights(layer, do_copy_weights)
        elif (input_size is not None) and (hidden_size is not None):
            self.layer = SkipGRULayer(input_size, hidden_size)
        else:
            raise Exception('Either layer or input_size & hidden_size are required')

    def impute_weights(self, donor_layer, copy=False):
        """
        Get weights from instance of nn.GRU.
        Args:
            donor_layer (nn.GRU): pretrained layer to get weights from.
            copy (bool): if True, will make copy of weights instead of linkage.
        """
        assert isinstance(donor_layer, nn.GRU), 'Wrong type of donor layer. GRU required!'

        if copy:
            self.layer.cell.inner_gru.weight_ih.data = donor_layer.weight_ih_l0.clone()
            self.layer.cell.inner_gru.weight_hh.data = donor_layer.weight_hh_l0.clone()
            self.layer.cell.inner_gru.bias_ih.data = donor_layer.bias_ih_l0.clone()
            self.layer.cell.inner_gru.bias_hh.data = donor_layer.bias_hh_l0.clone()
        else:
            self.layer.cell.inner_gru.weight_ih = donor_layer.weight_ih_l0
            self.layer.cell.inner_gru.weight_hh = donor_layer.weight_hh_l0
            self.layer.cell.inner_gru.bias_ih = donor_layer.bias_ih_l0
            self.layer.cell.inner_gru.bias_hh = donor_layer.bias_hh_l0

    def forward(self, x, b, l=None):
        """
        Make forward SkipGRU pass.
        Args:
            x (torch.FloatTensor): input of size [BxTxN]. NB! sequences should be padded from the end, not from the start position.
            u (torch.FloatTensor): mixture coefficients of size [BxTx1]
            l (torch.LongTensor): lengths of the padded sequences. If not provided, output will contain last elements of sequences.
        Returns:
            h (torch.FloatTensor, nn.utils.rnn.PackedSequence): history of hidden states of size [BxTxH]
            o (torch.FloatTensor): last hidden state of size [BxH]
        """
        h, o = self.layer(x, b)
        if l is not None:
            o = h[torch.arange(h.shape[0]), l-1]
        return h, o

In [5]:
import torch
import numpy as np
from torch import nn
#from skiprnn import SkipGRU
import torch.functional as F

class IMDbRNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_classes):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = SkipGRU(embedding_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)
        self.emb = nn.Embedding(vocab_size, embedding_size)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
        x, l = self.prepare_sequence(x)
        #на данном этапе учим с маской из одних единиц
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        return self.cls(o)

In [6]:
device = torch.device('cpu')
model = IMDbRNN(2000, 32, 16, 1)
model.to(device)

IMDbRNN(
  (rnn): SkipGRU(
    (layer): SkipGRULayer(
      original_name=SkipGRULayer
      (cell): SkipGRUCell(
        original_name=SkipGRUCell
        (inner_gru): ScriptModule(original_name=GRUCell)
      )
    )
  )
  (cls): Linear(in_features=16, out_features=1, bias=True)
  (emb): Embedding(2000, 32)
)

In [7]:
import torch
import torch.nn as nn
from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import AUCCallback, F1ScoreCallback, CheckpointCallback, CriterionCallback, CriterionAggregatorCallback

/Users/ayunts/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [8]:
num_epochs = 100
logdir = "./logs/exp3"
loaders = {
    "train": train_loader,
    "valid": val_loader
}

criterion = {
   "bce": nn.BCEWithLogitsLoss()
}

optimizer = torch.optim.Adam([
    {'params': model.parameters(), 'lr': 3e-4}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner()

In [8]:
runner.train(
    model=model,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_bce",
                     criterion_key='bce', multiplier=1.),
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_bce']),
               AUCCallback(num_classes=1), 
                F1ScoreCallback(),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='auc/_mean',
    minimize_metric=False,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 13:52:43,204] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=738.5268 | _timers/batch_time=0.7703 | _timers/data_time=0.0107 | _timers/model_time=0.7595 | auc/_mean=0.5220 | auc/class_0=0.5220 | f1_score=0.5011 | loss=0.6922 | loss_bce=0.6922
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=1178.0794 | _timers/batch_time=0.9482 | _timers/data_time=0.0192 | _timers/model_time=0.9290 | auc/_mean=0.5301 | auc/class_0=0.5301 | f1_score=0.3597 | loss=0.6911 | loss_bce=0.6911


KeyboardInterrupt: 

In [9]:
state = torch.load("best_skiprnn.pth",map_location='cpu')

model.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

In [10]:
model.eval()

IMDbRNN(
  (rnn): SkipGRU(
    (layer): SkipGRULayer(
      original_name=SkipGRULayer
      (cell): SkipGRUCell(
        original_name=SkipGRUCell
        (inner_gru): ScriptModule(original_name=GRUCell)
      )
    )
  )
  (cls): Linear(in_features=16, out_features=1, bias=True)
  (emb): Embedding(2000, 32)
)

In [13]:
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score



In [14]:
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        predictions.append(model(x).detach().cpu().numpy())
        labels.append(y.detach().cpu().numpy())
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

100%|██████████| 25/25 [00:21<00:00,  1.14it/s]


0.7049218304

## HintonBinarizer

In [15]:
from binarizers import HintonBinarizer

In [16]:
hb = HintonBinarizer()

In [17]:
for x,y in train_loader:
    break

In [18]:
x[0].shape

torch.Size([1306])

In [19]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        
        #считаем с бинаризованной маской u
        b_new = torch.repeat_interleave(hb(self.u), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [20]:
model_bin = IMDbRNN_bin(x[0].shape[0])

Делаем датасет из предложения x[0] размножив его несколько раз

In [21]:
from torch.utils.data import Dataset as BaseDataset

class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)


In [22]:
num_epochs = 100  # change me
logdir = "./logs/Hinton"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [ ]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        predictions.append(model(x).detach().cpu().numpy())
        labels.append(y.detach().cpu().numpy())
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

## ConcreteBinarizer

In [22]:
from binarizers import ConcreteBinarizer

In [23]:
cb = ConcreteBinarizer()

In [24]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
        
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        
        #считаем с бинаризованной маской u
        b_new = torch.repeat_interleave(cb(self.u), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [25]:
model_bin = IMDbRNN_bin(x[0].shape[0])

In [26]:
from torch.utils.data import Dataset as BaseDataset

class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)



In [27]:
num_epochs = 100  # change me
logdir = "./logs/Concrete"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [28]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 14:13:25,455] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=3.8579 | _timers/batch_time=0.5252 | _timers/data_time=0.0003 | _timers/model_time=0.5249 | f1_score=7.989e-07 | loss=0.1446 | loss_dist=0.0188 | loss_mean=0.1258
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=7.5091 | _timers/batch_time=0.2737 | _timers/data_time=0.0004 | _timers/model_time=0.2732 | f1_score=7.960e-07 | loss=0.1438 | loss_dist=0.0185 | loss_mean=0.1252
[2019-12-06 14:13:31,604] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=4.0934 | _timers/batch_time=0.4908 | _timers/data_time=0.0004 | _timers/model_time=0.4903 | f1_score=8.637e-07 | loss=0.2277 | loss_dist=0.1037 | loss_mean=0.1240
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=7.5170 | _timers/batch_time=0.2740 | _timers/data_time=0.0003 | _timers/model_time=0.2737 | f1_score=8.614e-07 | loss=0.2241 | loss_dist=

[2019-12-06 14:15:09,931] 
17/100 * Epoch 17 (train): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=3.9884 | _timers/batch_time=0.5034 | _timers/data_time=0.0005 | _timers/model_time=0.5029 | f1_score=9.102e-07 | loss=0.1491 | loss_dist=0.0420 | loss_mean=0.1071
17/100 * Epoch 17 (valid): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=7.3365 | _timers/batch_time=0.2793 | _timers/data_time=0.0003 | _timers/model_time=0.2789 | f1_score=9.111e-07 | loss=0.1477 | loss_dist=0.0412 | loss_mean=0.1065
[2019-12-06 14:15:16,003] 
18/100 * Epoch 18 (train): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=4.1062 | _timers/batch_time=0.4884 | _timers/data_time=0.0004 | _timers/model_time=0.4880 | f1_score=8.767e-07 | loss=0.1409 | loss_dist=0.0365 | loss_mean=0.1043
18/100 * Epoch 18 (valid): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=7.5171 | _timers/batch_time=0.2719 | _timers/data_time=0.0005 | _timers/model_time=0.2714 | f1_score=8.823e-07 | loss=0.1402 | lo

[2019-12-06 14:16:48,765] 
33/100 * Epoch 33 (train): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=4.1765 | _timers/batch_time=0.4810 | _timers/data_time=0.0004 | _timers/model_time=0.4806 | f1_score=7.755e-07 | loss=0.4404 | loss_dist=0.3475 | loss_mean=0.0930
33/100 * Epoch 33 (valid): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=7.5294 | _timers/batch_time=0.2712 | _timers/data_time=0.0004 | _timers/model_time=0.2708 | f1_score=7.754e-07 | loss=0.4374 | loss_dist=0.3449 | loss_mean=0.0925
[2019-12-06 14:16:55,098] 
34/100 * Epoch 34 (train): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=3.9050 | _timers/batch_time=0.5127 | _timers/data_time=0.0003 | _timers/model_time=0.5124 | f1_score=9.859e-07 | loss=0.2904 | loss_dist=0.2011 | loss_mean=0.0892
34/100 * Epoch 34 (valid): _base/lr=0.0016 | _base/momentum=0.9000 | _timers/_fps=7.6123 | _timers/batch_time=0.2702 | _timers/data_time=0.0002 | _timers/model_time=0.2700 | f1_score=9.862e-07 | loss=0.2882 | lo

[2019-12-06 14:18:32,638] 
49/100 * Epoch 49 (train): _base/lr=0.0014 | _base/momentum=0.9000 | _timers/_fps=3.5795 | _timers/batch_time=0.5731 | _timers/data_time=0.0003 | _timers/model_time=0.5728 | f1_score=1.146e-06 | loss=0.2485 | loss_dist=0.1659 | loss_mean=0.0826
49/100 * Epoch 49 (valid): _base/lr=0.0014 | _base/momentum=0.9000 | _timers/_fps=5.8738 | _timers/batch_time=0.3439 | _timers/data_time=0.0004 | _timers/model_time=0.3435 | f1_score=1.145e-06 | loss=0.2478 | loss_dist=0.1656 | loss_mean=0.0822
[2019-12-06 14:18:38,925] 
50/100 * Epoch 50 (train): _base/lr=0.0014 | _base/momentum=0.9000 | _timers/_fps=4.0628 | _timers/batch_time=0.4924 | _timers/data_time=0.0003 | _timers/model_time=0.4921 | f1_score=1.197e-06 | loss=0.1299 | loss_dist=0.0477 | loss_mean=0.0822
50/100 * Epoch 50 (valid): _base/lr=0.0014 | _base/momentum=0.9000 | _timers/_fps=6.9534 | _timers/batch_time=0.2915 | _timers/data_time=0.0002 | _timers/model_time=0.2913 | f1_score=1.197e-06 | loss=0.1295 | lo

[2019-12-06 14:20:12,031] 
65/100 * Epoch 65 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=4.1729 | _timers/batch_time=0.4815 | _timers/data_time=0.0004 | _timers/model_time=0.4810 | f1_score=8.477e-07 | loss=0.1204 | loss_dist=0.0466 | loss_mean=0.0738
65/100 * Epoch 65 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=7.5795 | _timers/batch_time=0.2704 | _timers/data_time=0.0005 | _timers/model_time=0.2699 | f1_score=8.475e-07 | loss=0.1201 | loss_dist=0.0465 | loss_mean=0.0736
[2019-12-06 14:20:18,041] 
66/100 * Epoch 66 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=4.2281 | _timers/batch_time=0.4749 | _timers/data_time=0.0003 | _timers/model_time=0.4746 | f1_score=8.578e-07 | loss=0.1669 | loss_dist=0.0936 | loss_mean=0.0733
66/100 * Epoch 66 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=7.5108 | _timers/batch_time=0.2722 | _timers/data_time=0.0003 | _timers/model_time=0.2719 | f1_score=8.575e-07 | loss=0.1661 | lo

[2019-12-06 14:21:54,108] 
81/100 * Epoch 81 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=4.0369 | _timers/batch_time=0.4960 | _timers/data_time=0.0003 | _timers/model_time=0.4957 | f1_score=9.333e-07 | loss=0.1457 | loss_dist=0.0754 | loss_mean=0.0703
81/100 * Epoch 81 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=7.6086 | _timers/batch_time=0.2684 | _timers/data_time=0.0002 | _timers/model_time=0.2681 | f1_score=9.323e-07 | loss=0.1453 | loss_dist=0.0752 | loss_mean=0.0701
[2019-12-06 14:22:00,231] 
82/100 * Epoch 82 (train): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=4.1170 | _timers/batch_time=0.4884 | _timers/data_time=0.0004 | _timers/model_time=0.4879 | f1_score=1.142e-06 | loss=0.1480 | loss_dist=0.0788 | loss_mean=0.0692
82/100 * Epoch 82 (valid): _base/lr=0.0008 | _base/momentum=0.9000 | _timers/_fps=7.3719 | _timers/batch_time=0.2778 | _timers/data_time=0.0003 | _timers/model_time=0.2774 | f1_score=1.141e-06 | loss=0.1475 | lo

[2019-12-06 14:23:40,315] 
97/100 * Epoch 97 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=3.7927 | _timers/batch_time=0.5281 | _timers/data_time=0.0003 | _timers/model_time=0.5277 | f1_score=9.343e-07 | loss=0.1554 | loss_dist=0.0890 | loss_mean=0.0664
97/100 * Epoch 97 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=6.9326 | _timers/batch_time=0.2940 | _timers/data_time=0.0003 | _timers/model_time=0.2936 | f1_score=9.344e-07 | loss=0.1551 | loss_dist=0.0889 | loss_mean=0.0663
[2019-12-06 14:23:47,115] 
98/100 * Epoch 98 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=3.7188 | _timers/batch_time=0.5387 | _timers/data_time=0.0003 | _timers/model_time=0.5384 | f1_score=1.010e-06 | loss=0.1015 | loss_dist=0.0376 | loss_mean=0.0639
98/100 * Epoch 98 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=7.0544 | _timers/batch_time=0.2892 | _timers/data_time=0.0003 | _timers/model_time=0.2889 | f1_score=1.010e-06 | loss=0.1014 | lo

In [ ]:
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        predictions.append(model(x).detach().cpu().numpy())
        labels.append(y.detach().cpu().numpy())
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

### HardKumaBinarizer

In [65]:
from binarizers import HardKumaBinarizer

In [30]:
hkb = HardKumaBinarizer()

In [31]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
        self.v = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)
        
        #считаем с бинаризованной маской u
        b_new = torch.repeat_interleave(hkb(self.u, self.v), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [32]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [33]:
num_epochs = 100  # change me
logdir = "./logs/HardKuma"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [34]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 14:31:51,203] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=3.4471 | _timers/batch_time=0.5825 | _timers/data_time=0.0006 | _timers/model_time=0.5818 | f1_score=4.872e-07 | loss=0.1452 | loss_dist=0.0107 | loss_mean=0.1345
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=6.4924 | _timers/batch_time=0.3149 | _timers/data_time=0.0004 | _timers/model_time=0.3145 | f1_score=4.945e-07 | loss=0.1435 | loss_dist=0.0096 | loss_mean=0.1339
[2019-12-06 14:31:57,825] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=4.0716 | _timers/batch_time=0.4921 | _timers/data_time=0.0004 | _timers/model_time=0.4916 | f1_score=4.415e-07 | loss=0.2597 | loss_dist=0.1240 | loss_mean=0.1357
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=6.2771 | _timers/batch_time=0.3261 | _timers/data_time=0.0004 | _timers/model_time=0.3257 | f1_score=4.463e-07 | loss=0.2484 | loss_dist=

[2019-12-06 14:33:36,085] 
17/100 * Epoch 17 (train): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=3.9140 | _timers/batch_time=0.5151 | _timers/data_time=0.0004 | _timers/model_time=0.5147 | f1_score=6.793e-07 | loss=0.1193 | loss_dist=0.0039 | loss_mean=0.1154
17/100 * Epoch 17 (valid): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=6.7095 | _timers/batch_time=0.3036 | _timers/data_time=0.0003 | _timers/model_time=0.3033 | f1_score=6.904e-07 | loss=0.1187 | loss_dist=0.0040 | loss_mean=0.1147
[2019-12-06 14:33:42,763] 
18/100 * Epoch 18 (train): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=3.7940 | _timers/batch_time=0.5293 | _timers/data_time=0.0004 | _timers/model_time=0.5288 | f1_score=8.228e-07 | loss=0.1283 | loss_dist=0.0113 | loss_mean=0.1170
18/100 * Epoch 18 (valid): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=6.7199 | _timers/batch_time=0.3041 | _timers/data_time=0.0004 | _timers/model_time=0.3037 | f1_score=8.304e-07 | loss=0.1278 | lo

[2019-12-06 14:35:20,472] 
33/100 * Epoch 33 (train): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=4.0691 | _timers/batch_time=0.4934 | _timers/data_time=0.0004 | _timers/model_time=0.4929 | f1_score=7.436e-07 | loss=0.1098 | loss_dist=0.0093 | loss_mean=0.1005
33/100 * Epoch 33 (valid): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=7.4506 | _timers/batch_time=0.2753 | _timers/data_time=0.0004 | _timers/model_time=0.2749 | f1_score=7.482e-07 | loss=0.1090 | loss_dist=0.0092 | loss_mean=0.0998
[2019-12-06 14:35:26,677] 
34/100 * Epoch 34 (train): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=4.0438 | _timers/batch_time=0.4960 | _timers/data_time=0.0003 | _timers/model_time=0.4957 | f1_score=9.816e-07 | loss=0.1110 | loss_dist=0.0140 | loss_mean=0.0970
34/100 * Epoch 34 (valid): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=7.4801 | _timers/batch_time=0.2738 | _timers/data_time=0.0004 | _timers/model_time=0.2734 | f1_score=9.864e-07 | loss=0.1107 | lo

[2019-12-06 14:37:04,135] 
49/100 * Epoch 49 (train): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=4.0168 | _timers/batch_time=0.5002 | _timers/data_time=0.0002 | _timers/model_time=0.5000 | f1_score=1.048e-06 | loss=0.1118 | loss_dist=0.0273 | loss_mean=0.0846
49/100 * Epoch 49 (valid): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=6.9993 | _timers/batch_time=0.2944 | _timers/data_time=0.0002 | _timers/model_time=0.2941 | f1_score=1.048e-06 | loss=0.1111 | loss_dist=0.0271 | loss_mean=0.0840
[2019-12-06 14:37:10,795] 
50/100 * Epoch 50 (train): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=3.8346 | _timers/batch_time=0.5254 | _timers/data_time=0.0003 | _timers/model_time=0.5250 | f1_score=1.194e-06 | loss=0.1129 | loss_dist=0.0299 | loss_mean=0.0830
50/100 * Epoch 50 (valid): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=6.9160 | _timers/batch_time=0.2971 | _timers/data_time=0.0003 | _timers/model_time=0.2968 | f1_score=1.194e-06 | loss=0.1122 | lo

[2019-12-06 14:38:48,999] 
65/100 * Epoch 65 (train): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=3.9446 | _timers/batch_time=0.5080 | _timers/data_time=0.0004 | _timers/model_time=0.5075 | f1_score=9.539e-07 | loss=0.0786 | loss_dist=0.0091 | loss_mean=0.0695
65/100 * Epoch 65 (valid): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=7.3104 | _timers/batch_time=0.2813 | _timers/data_time=0.0003 | _timers/model_time=0.2810 | f1_score=9.528e-07 | loss=0.0780 | loss_dist=0.0090 | loss_mean=0.0691
[2019-12-06 14:38:55,271] 
66/100 * Epoch 66 (train): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=4.0508 | _timers/batch_time=0.4952 | _timers/data_time=0.0004 | _timers/model_time=0.4947 | f1_score=9.110e-07 | loss=0.0781 | loss_dist=0.0073 | loss_mean=0.0709
66/100 * Epoch 66 (valid): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=7.4252 | _timers/batch_time=0.2763 | _timers/data_time=0.0003 | _timers/model_time=0.2759 | f1_score=9.106e-07 | loss=0.0776 | lo

[2019-12-06 14:40:38,610] 
81/100 * Epoch 81 (train): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=3.5342 | _timers/batch_time=0.5815 | _timers/data_time=0.0004 | _timers/model_time=0.5811 | f1_score=9.736e-07 | loss=0.0721 | loss_dist=0.0120 | loss_mean=0.0601
81/100 * Epoch 81 (valid): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=6.6562 | _timers/batch_time=0.3098 | _timers/data_time=0.0003 | _timers/model_time=0.3095 | f1_score=9.712e-07 | loss=0.0716 | loss_dist=0.0119 | loss_mean=0.0597
[2019-12-06 14:40:45,436] 
82/100 * Epoch 82 (train): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=3.8152 | _timers/batch_time=0.5255 | _timers/data_time=0.0004 | _timers/model_time=0.5251 | f1_score=1.008e-06 | loss=0.0741 | loss_dist=0.0133 | loss_mean=0.0608
82/100 * Epoch 82 (valid): _base/lr=0.0022 | _base/momentum=0.9000 | _timers/_fps=6.8262 | _timers/batch_time=0.3004 | _timers/data_time=0.0003 | _timers/model_time=0.3001 | f1_score=1.006e-06 | loss=0.0736 | lo

[2019-12-06 14:42:24,217] 
97/100 * Epoch 97 (train): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=3.8758 | _timers/batch_time=0.5197 | _timers/data_time=0.0003 | _timers/model_time=0.5194 | f1_score=9.981e-07 | loss=0.0640 | loss_dist=0.0131 | loss_mean=0.0509
97/100 * Epoch 97 (valid): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=7.0581 | _timers/batch_time=0.2893 | _timers/data_time=0.0005 | _timers/model_time=0.2888 | f1_score=9.960e-07 | loss=0.0635 | loss_dist=0.0128 | loss_mean=0.0506
[2019-12-06 14:42:30,788] 
98/100 * Epoch 98 (train): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=3.8946 | _timers/batch_time=0.5142 | _timers/data_time=0.0004 | _timers/model_time=0.5137 | f1_score=8.008e-07 | loss=0.0522 | loss_dist=0.0022 | loss_mean=0.0500
98/100 * Epoch 98 (valid): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=7.1488 | _timers/batch_time=0.2862 | _timers/data_time=0.0002 | _timers/model_time=0.2859 | f1_score=7.989e-07 | loss=0.0519 | lo

In [ ]:
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        predictions.append(model(x).detach().cpu().numpy())
        labels.append(y.detach().cpu().numpy())
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

### Bernoulli p = 0.5

In [124]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)               
        
        #считаем с бинаризованной маской u
        #torch.bernoulli(torch.tensor([1.0]),)
        b_new = torch.bernoulli(torch.ones(x.shape[:2]),0.5).unsqueeze(2).to(self.emb.weight.device)
        #b_new = torch.repeat_interleave(bb(0.4), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [125]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [126]:
num_epochs = 100  # change me
logdir = "./logs/b50"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [127]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 16:24:44,693] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=4.0558 | _timers/batch_time=0.4958 | _timers/data_time=0.0002 | _timers/model_time=0.4956 | f1_score=4.793e-07 | loss=0.4527 | loss_dist=0.3225 | loss_mean=0.1302
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=6.8390 | _timers/batch_time=0.2968 | _timers/data_time=0.0002 | _timers/model_time=0.2966 | f1_score=4.793e-07 | loss=0.4527 | loss_dist=0.3225 | loss_mean=0.1302
[2019-12-06 16:24:44,693] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=4.0558 | _timers/batch_time=0.4958 | _timers/data_time=0.0002 | _timers/model_time=0.4956 | f1_score=4.793e-07 | loss=0.4527 | loss_dist=0.3225 | loss_mean=0.1302
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=6.8390 | _timers/batch_time=0.2968 | _timers/data_time=0.0002 | _timers/model_time=0.2966 | f1_score=4.793e-07 | loss=0.4527 | loss_dist=

[2019-12-06 16:24:51,416] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=3.7502 | _timers/batch_time=0.5402 | _timers/data_time=0.0002 | _timers/model_time=0.5400 | f1_score=5.540e-07 | loss=0.2722 | loss_dist=0.1504 | loss_mean=0.1218
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=6.6894 | _timers/batch_time=0.3049 | _timers/data_time=0.0003 | _timers/model_time=0.3046 | f1_score=5.540e-07 | loss=0.2722 | loss_dist=0.1504 | loss_mean=0.1218
[2019-12-06 16:24:51,416] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=3.7502 | _timers/batch_time=0.5402 | _timers/data_time=0.0002 | _timers/model_time=0.5400 | f1_score=5.540e-07 | loss=0.2722 | loss_dist=0.1504 | loss_mean=0.1218
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=6.6894 | _timers/batch_time=0.3049 | _timers/data_time=0.0003 | _timers/model_time=0.3046 | f1_score=5.540e-07 | loss=0.2722 | loss_dist=

...

[2019-12-06 16:34:34,268] 
93/100 * Epoch 93 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=3.8435 | _timers/batch_time=0.5234 | _timers/data_time=0.0003 | _timers/model_time=0.5230 | f1_score=4.916e-07 | loss=0.3562 | loss_dist=0.2303 | loss_mean=0.1259
93/100 * Epoch 93 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=6.0780 | _timers/batch_time=0.3387 | _timers/data_time=0.0003 | _timers/model_time=0.3384 | f1_score=4.916e-07 | loss=0.3562 | loss_dist=0.2303 | loss_mean=0.1259
[2019-12-06 16:34:34,268] 
93/100 * Epoch 93 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=3.8435 | _timers/batch_time=0.5234 | _timers/data_time=0.0003 | _timers/model_time=0.5230 | f1_score=4.916e-07 | loss=0.3562 | loss_dist=0.2303 | loss_mean=0.1259
93/100 * Epoch 93 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=6.0780 | _timers/batch_time=0.3387 | _timers/data_time=0.0003 | _timers/model_time=0.3384 | f1_score=4.916e-07 | loss=0.3562 | lo

[2019-12-06 16:34:48,596] 
95/100 * Epoch 95 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=3.6077 | _timers/batch_time=0.5554 | _timers/data_time=0.0004 | _timers/model_time=0.5550 | f1_score=4.661e-07 | loss=0.7140 | loss_dist=0.5888 | loss_mean=0.1252
95/100 * Epoch 95 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=6.7003 | _timers/batch_time=0.3038 | _timers/data_time=0.0003 | _timers/model_time=0.3035 | f1_score=4.661e-07 | loss=0.7140 | loss_dist=0.5888 | loss_mean=0.1252
[2019-12-06 16:34:48,596] 
95/100 * Epoch 95 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=3.6077 | _timers/batch_time=0.5554 | _timers/data_time=0.0004 | _timers/model_time=0.5550 | f1_score=4.661e-07 | loss=0.7140 | loss_dist=0.5888 | loss_mean=0.1252
95/100 * Epoch 95 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=6.7003 | _timers/batch_time=0.3038 | _timers/data_time=0.0003 | _timers/model_time=0.3035 | f1_score=4.661e-07 | loss=0.7140 | lo

[2019-12-06 16:35:02,107] 
97/100 * Epoch 97 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=3.6762 | _timers/batch_time=0.5459 | _timers/data_time=0.0004 | _timers/model_time=0.5454 | f1_score=5.754e-07 | loss=0.8442 | loss_dist=0.7203 | loss_mean=0.1240
97/100 * Epoch 97 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=7.1804 | _timers/batch_time=0.2829 | _timers/data_time=0.0003 | _timers/model_time=0.2826 | f1_score=5.754e-07 | loss=0.8442 | loss_dist=0.7203 | loss_mean=0.1240
[2019-12-06 16:35:02,107] 
97/100 * Epoch 97 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=3.6762 | _timers/batch_time=0.5459 | _timers/data_time=0.0004 | _timers/model_time=0.5454 | f1_score=5.754e-07 | loss=0.8442 | loss_dist=0.7203 | loss_mean=0.1240
97/100 * Epoch 97 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=7.1804 | _timers/batch_time=0.2829 | _timers/data_time=0.0003 | _timers/model_time=0.2826 | f1_score=5.754e-07 | loss=0.8442 | lo

[2019-12-06 16:35:08,343] 
98/100 * Epoch 98 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=4.0180 | _timers/batch_time=0.5008 | _timers/data_time=0.0003 | _timers/model_time=0.5004 | f1_score=6.901e-07 | loss=0.4667 | loss_dist=0.3436 | loss_mean=0.1232
98/100 * Epoch 98 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=7.6691 | _timers/batch_time=0.2660 | _timers/data_time=0.0003 | _timers/model_time=0.2656 | f1_score=6.901e-07 | loss=0.4667 | loss_dist=0.3436 | loss_mean=0.1232
[2019-12-06 16:35:08,343] 
98/100 * Epoch 98 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=4.0180 | _timers/batch_time=0.5008 | _timers/data_time=0.0003 | _timers/model_time=0.5004 | f1_score=6.901e-07 | loss=0.4667 | loss_dist=0.3436 | loss_mean=0.1232
98/100 * Epoch 98 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=7.6691 | _timers/batch_time=0.2660 | _timers/data_time=0.0003 | _timers/model_time=0.2656 | f1_score=6.901e-07 | loss=0.4667 | lo

[2019-12-06 16:35:20,328] 
100/100 * Epoch 100 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=4.2947 | _timers/batch_time=0.4672 | _timers/data_time=0.0005 | _timers/model_time=0.4667 | f1_score=6.500e-07 | loss=0.2726 | loss_dist=0.1469 | loss_mean=0.1257
100/100 * Epoch 100 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=7.6328 | _timers/batch_time=0.2661 | _timers/data_time=0.0002 | _timers/model_time=0.2658 | f1_score=6.500e-07 | loss=0.2726 | loss_dist=0.1469 | loss_mean=0.1257
[2019-12-06 16:35:20,328] 
100/100 * Epoch 100 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=4.2947 | _timers/batch_time=0.4672 | _timers/data_time=0.0005 | _timers/model_time=0.4667 | f1_score=6.500e-07 | loss=0.2726 | loss_dist=0.1469 | loss_mean=0.1257
100/100 * Epoch 100 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=7.6328 | _timers/batch_time=0.2661 | _timers/data_time=0.0002 | _timers/model_time=0.2658 | f1_score=6.500e-07 | loss=0.2

In [ ]:
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        predictions.append(model(x).detach().cpu().numpy())
        labels.append(y.detach().cpu().numpy())
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

### Bernoulli p = 0.9

In [128]:
p = 0.9

In [129]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)               
        
        #считаем с бинаризованной маской u
        #torch.bernoulli(torch.tensor([1.0]),)
        b_new = torch.bernoulli(torch.ones(x.shape[:2]),p).unsqueeze(2).to(self.emb.weight.device)
        #b_new = torch.repeat_interleave(bb(0.4), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [130]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [131]:
num_epochs = 100  # change me
logdir = "./logs/b90"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [132]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 16:43:11,703] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=3.8629 | _timers/batch_time=0.5237 | _timers/data_time=0.0003 | _timers/model_time=0.5233 | f1_score=6.329e-07 | loss=0.3451 | loss_dist=0.0238 | loss_mean=0.3212
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=7.5049 | _timers/batch_time=0.2712 | _timers/data_time=0.0005 | _timers/model_time=0.2707 | f1_score=6.329e-07 | loss=0.3451 | loss_dist=0.0238 | loss_mean=0.3212
[2019-12-06 16:43:18,544] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=3.8251 | _timers/batch_time=0.5297 | _timers/data_time=0.0002 | _timers/model_time=0.5294 | f1_score=5.918e-07 | loss=0.3369 | loss_dist=0.0212 | loss_mean=0.3157
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=6.8569 | _timers/batch_time=0.2960 | _timers/data_time=0.0004 | _timers/model_time=0.2956 | f1_score=5.918e-07 | loss=0.3369 | loss_dist=

[2019-12-06 16:45:01,528] 
17/100 * Epoch 17 (train): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=3.7539 | _timers/batch_time=0.5335 | _timers/data_time=0.0002 | _timers/model_time=0.5332 | f1_score=6.562e-07 | loss=0.4500 | loss_dist=0.1262 | loss_mean=0.3238
17/100 * Epoch 17 (valid): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=6.5200 | _timers/batch_time=0.3120 | _timers/data_time=0.0004 | _timers/model_time=0.3115 | f1_score=6.562e-07 | loss=0.4500 | loss_dist=0.1262 | loss_mean=0.3238
[2019-12-06 16:45:08,433] 
18/100 * Epoch 18 (train): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=3.7106 | _timers/batch_time=0.5406 | _timers/data_time=0.0004 | _timers/model_time=0.5402 | f1_score=6.423e-07 | loss=0.3500 | loss_dist=0.0292 | loss_mean=0.3208
18/100 * Epoch 18 (valid): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=6.4085 | _timers/batch_time=0.3172 | _timers/data_time=0.0003 | _timers/model_time=0.3169 | f1_score=6.423e-07 | loss=0.3500 | lo

[2019-12-06 16:46:47,151] 
33/100 * Epoch 33 (train): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=3.8357 | _timers/batch_time=0.5220 | _timers/data_time=0.0003 | _timers/model_time=0.5216 | f1_score=4.882e-07 | loss=0.3635 | loss_dist=0.0419 | loss_mean=0.3216
33/100 * Epoch 33 (valid): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=7.0742 | _timers/batch_time=0.2882 | _timers/data_time=0.0004 | _timers/model_time=0.2878 | f1_score=4.882e-07 | loss=0.3635 | loss_dist=0.0419 | loss_mean=0.3216
[2019-12-06 16:46:53,664] 
34/100 * Epoch 34 (train): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=3.9831 | _timers/batch_time=0.5048 | _timers/data_time=0.0003 | _timers/model_time=0.5045 | f1_score=6.530e-07 | loss=0.3549 | loss_dist=0.0352 | loss_mean=0.3196
34/100 * Epoch 34 (valid): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=6.7478 | _timers/batch_time=0.3007 | _timers/data_time=0.0002 | _timers/model_time=0.3004 | f1_score=6.530e-07 | loss=0.3549 | lo

[2019-12-06 16:48:31,927] 
49/100 * Epoch 49 (train): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=3.9200 | _timers/batch_time=0.5121 | _timers/data_time=0.0003 | _timers/model_time=0.5118 | f1_score=6.040e-07 | loss=0.3543 | loss_dist=0.0413 | loss_mean=0.3130
49/100 * Epoch 49 (valid): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=6.7265 | _timers/batch_time=0.3002 | _timers/data_time=0.0003 | _timers/model_time=0.2998 | f1_score=6.040e-07 | loss=0.3543 | loss_dist=0.0413 | loss_mean=0.3130
[2019-12-06 16:48:38,501] 
50/100 * Epoch 50 (train): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=3.8640 | _timers/batch_time=0.5189 | _timers/data_time=0.0003 | _timers/model_time=0.5185 | f1_score=6.065e-07 | loss=0.4030 | loss_dist=0.0807 | loss_mean=0.3223
50/100 * Epoch 50 (valid): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=7.2494 | _timers/batch_time=0.2806 | _timers/data_time=0.0003 | _timers/model_time=0.2803 | f1_score=6.065e-07 | loss=0.4030 | lo

[2019-12-06 16:50:16,824] 
65/100 * Epoch 65 (train): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=3.7952 | _timers/batch_time=0.5283 | _timers/data_time=0.0003 | _timers/model_time=0.5279 | f1_score=6.276e-07 | loss=0.3690 | loss_dist=0.0461 | loss_mean=0.3229
65/100 * Epoch 65 (valid): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=7.0192 | _timers/batch_time=0.2907 | _timers/data_time=0.0004 | _timers/model_time=0.2903 | f1_score=6.276e-07 | loss=0.3690 | loss_dist=0.0461 | loss_mean=0.3229
[2019-12-06 16:50:23,395] 
66/100 * Epoch 66 (train): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=3.8686 | _timers/batch_time=0.5184 | _timers/data_time=0.0003 | _timers/model_time=0.5181 | f1_score=6.232e-07 | loss=0.3846 | loss_dist=0.0632 | loss_mean=0.3214
66/100 * Epoch 66 (valid): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=6.7778 | _timers/batch_time=0.2999 | _timers/data_time=0.0002 | _timers/model_time=0.2997 | f1_score=6.232e-07 | loss=0.3846 | lo

[2019-12-06 16:51:58,455] 
81/100 * Epoch 81 (train): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=4.1329 | _timers/batch_time=0.4868 | _timers/data_time=0.0004 | _timers/model_time=0.4863 | f1_score=5.643e-07 | loss=0.3747 | loss_dist=0.0554 | loss_mean=0.3193
81/100 * Epoch 81 (valid): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=7.6555 | _timers/batch_time=0.2649 | _timers/data_time=0.0004 | _timers/model_time=0.2645 | f1_score=5.643e-07 | loss=0.3747 | loss_dist=0.0554 | loss_mean=0.3193
[2019-12-06 16:52:04,424] 
82/100 * Epoch 82 (train): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=4.2761 | _timers/batch_time=0.4697 | _timers/data_time=0.0003 | _timers/model_time=0.4693 | f1_score=7.101e-07 | loss=0.3966 | loss_dist=0.0794 | loss_mean=0.3172
82/100 * Epoch 82 (valid): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=7.4502 | _timers/batch_time=0.2736 | _timers/data_time=0.0002 | _timers/model_time=0.2734 | f1_score=7.101e-07 | loss=0.3966 | lo

[2019-12-06 16:53:41,393] 
97/100 * Epoch 97 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=3.8754 | _timers/batch_time=0.5198 | _timers/data_time=0.0002 | _timers/model_time=0.5195 | f1_score=5.737e-07 | loss=0.3501 | loss_dist=0.0321 | loss_mean=0.3180
97/100 * Epoch 97 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=6.9574 | _timers/batch_time=0.2918 | _timers/data_time=0.0003 | _timers/model_time=0.2915 | f1_score=5.737e-07 | loss=0.3501 | loss_dist=0.0321 | loss_mean=0.3180
[2019-12-06 16:53:48,017] 
98/100 * Epoch 98 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=3.8625 | _timers/batch_time=0.5188 | _timers/data_time=0.0002 | _timers/model_time=0.5186 | f1_score=5.898e-07 | loss=0.3512 | loss_dist=0.0345 | loss_mean=0.3167
98/100 * Epoch 98 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=6.7303 | _timers/batch_time=0.3020 | _timers/data_time=0.0004 | _timers/model_time=0.3016 | f1_score=5.898e-07 | loss=0.3512 | lo

In [ ]:
predictions, labels = [], []
with torch.no_grad():
    for x,y in tqdm(test_loader):
        predictions.append(model(x).detach().cpu().numpy())
        labels.append(y.detach().cpu().numpy())
predictions = np.concatenate(predictions)
labels = np.concatenate(labels)

roc_auc_score(labels, predictions)

### Bernoulli p = 0.2

In [133]:
p = 0.2

In [134]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)               
        
        #считаем с бинаризованной маской u
        #torch.bernoulli(torch.tensor([1.0]),)
        b_new = torch.bernoulli(torch.ones(x.shape[:2]),p).unsqueeze(2).to(self.emb.weight.device)
        #b_new = torch.repeat_interleave(bb(0.4), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [135]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [136]:
num_epochs = 100  # change me
logdir = "./logs/b20"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [137]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 16:56:20,750] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=3.7287 | _timers/batch_time=0.5383 | _timers/data_time=0.0002 | _timers/model_time=0.5381 | f1_score=4.387e-07 | loss=1.6459 | loss_dist=1.6250 | loss_mean=0.0210
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=7.0189 | _timers/batch_time=0.2942 | _timers/data_time=0.0003 | _timers/model_time=0.2939 | f1_score=4.387e-07 | loss=1.6459 | loss_dist=1.6250 | loss_mean=0.0210
[2019-12-06 16:56:27,166] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=3.9150 | _timers/batch_time=0.5114 | _timers/data_time=0.0006 | _timers/model_time=0.5108 | f1_score=4.680e-07 | loss=1.6268 | loss_dist=1.6071 | loss_mean=0.0197
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=7.0276 | _timers/batch_time=0.2886 | _timers/data_time=0.0003 | _timers/model_time=0.2883 | f1_score=4.680e-07 | loss=1.6268 | loss_dist=

[2019-12-06 16:58:05,515] 
17/100 * Epoch 17 (train): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=4.0059 | _timers/batch_time=0.5025 | _timers/data_time=0.0005 | _timers/model_time=0.5020 | f1_score=7.666e-07 | loss=2.1012 | loss_dist=2.0807 | loss_mean=0.0204
17/100 * Epoch 17 (valid): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=7.3927 | _timers/batch_time=0.2759 | _timers/data_time=0.0003 | _timers/model_time=0.2756 | f1_score=7.666e-07 | loss=2.1012 | loss_dist=2.0807 | loss_mean=0.0204
[2019-12-06 16:58:11,482] 
18/100 * Epoch 18 (train): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=4.1572 | _timers/batch_time=0.4823 | _timers/data_time=0.0003 | _timers/model_time=0.4819 | f1_score=4.589e-07 | loss=2.0422 | loss_dist=2.0215 | loss_mean=0.0206
18/100 * Epoch 18 (valid): _base/lr=0.0027 | _base/momentum=0.9000 | _timers/_fps=7.5995 | _timers/batch_time=0.2670 | _timers/data_time=0.0003 | _timers/model_time=0.2667 | f1_score=4.589e-07 | loss=2.0422 | lo

[2019-12-06 16:59:45,764] 
33/100 * Epoch 33 (train): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=4.0683 | _timers/batch_time=0.4942 | _timers/data_time=0.0003 | _timers/model_time=0.4938 | f1_score=5.646e-07 | loss=1.0204 | loss_dist=0.9993 | loss_mean=0.0211
33/100 * Epoch 33 (valid): _base/lr=0.0020 | _base/momentum=0.9000 | _timers/_fps=7.4399 | _timers/batch_time=0.2757 | _timers/data_time=0.0003 | _timers/model_time=0.2754 | f1_score=5.646e-07 | loss=1.0204 | loss_dist=0.9993 | loss_mean=0.0211
[2019-12-06 16:59:51,973] 
34/100 * Epoch 34 (train): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=4.0399 | _timers/batch_time=0.4982 | _timers/data_time=0.0003 | _timers/model_time=0.4978 | f1_score=4.245e-07 | loss=2.3611 | loss_dist=2.3414 | loss_mean=0.0197
34/100 * Epoch 34 (valid): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=7.0821 | _timers/batch_time=0.2875 | _timers/data_time=0.0003 | _timers/model_time=0.2871 | f1_score=4.245e-07 | loss=2.3611 | lo

[2019-12-06 17:01:30,574] 
49/100 * Epoch 49 (train): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=3.9633 | _timers/batch_time=0.5074 | _timers/data_time=0.0002 | _timers/model_time=0.5071 | f1_score=4.797e-07 | loss=2.5177 | loss_dist=2.4983 | loss_mean=0.0194
49/100 * Epoch 49 (valid): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=6.6143 | _timers/batch_time=0.3091 | _timers/data_time=0.0002 | _timers/model_time=0.3088 | f1_score=4.797e-07 | loss=2.5177 | loss_dist=2.4983 | loss_mean=0.0194
[2019-12-06 17:01:36,914] 
50/100 * Epoch 50 (train): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=4.0441 | _timers/batch_time=0.4952 | _timers/data_time=0.0002 | _timers/model_time=0.4949 | f1_score=4.398e-07 | loss=2.8425 | loss_dist=2.8216 | loss_mean=0.0208
50/100 * Epoch 50 (valid): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=7.0178 | _timers/batch_time=0.2910 | _timers/data_time=0.0003 | _timers/model_time=0.2906 | f1_score=4.398e-07 | loss=2.8425 | lo

[2019-12-06 17:03:15,903] 
65/100 * Epoch 65 (train): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=4.1155 | _timers/batch_time=0.4873 | _timers/data_time=0.0002 | _timers/model_time=0.4870 | f1_score=6.574e-07 | loss=1.6849 | loss_dist=1.6648 | loss_mean=0.0201
65/100 * Epoch 65 (valid): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=7.4148 | _timers/batch_time=0.2747 | _timers/data_time=0.0002 | _timers/model_time=0.2745 | f1_score=6.574e-07 | loss=1.6849 | loss_dist=1.6648 | loss_mean=0.0201
[2019-12-06 17:03:22,021] 
66/100 * Epoch 66 (train): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=4.1235 | _timers/batch_time=0.4869 | _timers/data_time=0.0004 | _timers/model_time=0.4865 | f1_score=6.502e-07 | loss=1.1841 | loss_dist=1.1646 | loss_mean=0.0195
66/100 * Epoch 66 (valid): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=7.4967 | _timers/batch_time=0.2712 | _timers/data_time=0.0003 | _timers/model_time=0.2709 | f1_score=6.502e-07 | loss=1.1841 | lo

[2019-12-06 17:04:59,073] 
81/100 * Epoch 81 (train): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=3.7275 | _timers/batch_time=0.5437 | _timers/data_time=0.0002 | _timers/model_time=0.5434 | f1_score=7.208e-07 | loss=3.3885 | loss_dist=3.3688 | loss_mean=0.0197
81/100 * Epoch 81 (valid): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=6.6554 | _timers/batch_time=0.3048 | _timers/data_time=0.0003 | _timers/model_time=0.3045 | f1_score=7.208e-07 | loss=3.3885 | loss_dist=3.3688 | loss_mean=0.0197
[2019-12-06 17:05:05,649] 
82/100 * Epoch 82 (train): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=3.8248 | _timers/batch_time=0.5242 | _timers/data_time=0.0003 | _timers/model_time=0.5239 | f1_score=4.404e-07 | loss=2.7759 | loss_dist=2.7549 | loss_mean=0.0211
82/100 * Epoch 82 (valid): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=7.1512 | _timers/batch_time=0.2850 | _timers/data_time=0.0002 | _timers/model_time=0.2847 | f1_score=4.404e-07 | loss=2.7759 | lo

[2019-12-06 17:06:44,027] 
97/100 * Epoch 97 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=3.8736 | _timers/batch_time=0.5170 | _timers/data_time=0.0004 | _timers/model_time=0.5166 | f1_score=4.665e-07 | loss=1.4659 | loss_dist=1.4471 | loss_mean=0.0188
97/100 * Epoch 97 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=6.7393 | _timers/batch_time=0.3038 | _timers/data_time=0.0002 | _timers/model_time=0.3035 | f1_score=4.665e-07 | loss=1.4659 | loss_dist=1.4471 | loss_mean=0.0188
[2019-12-06 17:06:50,851] 
98/100 * Epoch 98 (train): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=3.7237 | _timers/batch_time=0.5384 | _timers/data_time=0.0004 | _timers/model_time=0.5380 | f1_score=5.703e-07 | loss=1.9826 | loss_dist=1.9632 | loss_mean=0.0194
98/100 * Epoch 98 (valid): _base/lr=0.0006 | _base/momentum=0.9000 | _timers/_fps=6.6220 | _timers/batch_time=0.3083 | _timers/data_time=0.0003 | _timers/model_time=0.3080 | f1_score=5.703e-07 | loss=1.9826 | lo

### Bernoulli p = 0.8

In [128]:
p = 0.8

In [129]:
from torch.autograd import Variable


class IMDbRNN_bin(nn.Module):
    def __init__(self, sent_size):
        super().__init__()
#         self.rnn = nn.GRU(embedding_size, hidden_size)
        self.rnn = model.rnn
        self.cls = model.cls
        self.emb = model.emb
        
        self.u = Variable(torch.randn(1,sent_size,1), requires_grad=True)
    
    def prepare_sequence(self, x):
        l = torch.LongTensor([len(xi) for xi in x])
        x = torch.split_with_sizes(self.emb(torch.cat(x).to(self.emb.weight.device)), l.unbind(0))
        x = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=5)
        return x, l
    
    def forward(self, x):
#         print(x.shape,x[0].shape)
        
        x = [i for i in x]
        x, l = self.prepare_sequence(x)
        
        #считаем с единичной маской
        b = torch.ones(x.shape[:2]).unsqueeze(2).to(self.emb.weight.device)
        h,o = self.rnn(x,b,l)               
        
        #считаем с бинаризованной маской u
        #torch.bernoulli(torch.tensor([1.0]),)
        b_new = torch.bernoulli(torch.ones(x.shape[:2]),p).unsqueeze(2).to(self.emb.weight.device)
        #b_new = torch.repeat_interleave(bb(0.4), repeats=len(x), dim=0).to(self.emb.weight.device)
        h_new,o_new = self.rnn(x,b_new,l)
        
        #расстояние между выходами сети на полном и прореженом предложении
        dist = ((o-o_new)**2).sum(1)
        
        #среднее значение бинарной маски
        mean = b_new.squeeze(2).mean(1)
        
        #финальное предсказание класса на прореженом сообщении
        target = self.cls(o_new)
        
        return {'cls': target, 'dist': dist, 'mean': mean}

In [130]:
model_bin = IMDbRNN_bin(x[0].shape[0])
class Dataset(BaseDataset):

    
    def __init__(
            self, x, y, idx
    ):
        
        self.x = x
        self.y = y
        self.idx = idx
    
    def __getitem__(self, idx):

        return {'features': self.x[self.idx], 'target_cls':self.y[self.idx],
                'target_dist': torch.tensor(0).float(), 'target_mean': torch.tensor(0).float()}
        
    def __len__(self):
        return 8


new_train_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)
new_val_loader = DataLoader(Dataset(x,y,0), shuffle=True, batch_size=2)

In [131]:
num_epochs = 100  # change me
logdir = "./logs/HardKuma"
loaders = {
    "train": new_train_loader,
    "valid": new_val_loader
}

criterion = {
   "mse": nn.MSELoss()
}

optimizer = torch.optim.Adam([
    {'params': model_bin.u, 'lr': 3e-3}])


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=4)

runner = SupervisedRunner(input_target_key=None,output_key=None)

In [132]:
runner.train(
    model=model_bin,
    criterion=criterion,
    scheduler=scheduler,
    callbacks=[   
               CriterionCallback(prefix="loss_dist", input_key='target_dist', output_key='dist',
                     criterion_key='mse', multiplier=0.5),
               CriterionCallback(prefix="loss_mean", input_key='target_mean', output_key='mean',
                     criterion_key='mse', multiplier=0.5),
        
               CriterionAggregatorCallback(prefix="loss",loss_keys=['loss_dist','loss_mean']),
#                AUCCallback(num_classes=1, input_key='target_cls', output_key='cls'), 
                F1ScoreCallback(input_key='target_cls', output_key='cls'),
               CheckpointCallback(save_n_best=3)
                ],

    optimizer=optimizer,
    main_metric='loss',
    minimize_metric=True,
    loaders=loaders,
    logdir=logdir,
    num_epochs=num_epochs,
#     fp16={"opt_level": "O1"},
    verbose=False
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[2019-12-06 16:43:11,703] 
1/100 * Epoch 1 (train): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=3.8629 | _timers/batch_time=0.5237 | _timers/data_time=0.0003 | _timers/model_time=0.5233 | f1_score=6.329e-07 | loss=0.3451 | loss_dist=0.0238 | loss_mean=0.3212
1/100 * Epoch 1 (valid): _base/lr=0.0010 | _base/momentum=0.9000 | _timers/_fps=7.5049 | _timers/batch_time=0.2712 | _timers/data_time=0.0005 | _timers/model_time=0.2707 | f1_score=6.329e-07 | loss=0.3451 | loss_dist=0.0238 | loss_mean=0.3212
[2019-12-06 16:43:18,544] 
2/100 * Epoch 2 (train): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=3.8251 | _timers/batch_time=0.5297 | _timers/data_time=0.0002 | _timers/model_time=0.5294 | f1_score=5.918e-07 | loss=0.3369 | loss_dist=0.0212 | loss_mean=0.3157
2/100 * Epoch 2 (valid): _base/lr=0.0030 | _base/momentum=0.9000 | _timers/_fps=6.8569 | _timers/batch_time=0.2960 | _timers/data_time=0.0004 | _timers/model_time=0.2956 | f1_score=5.918e-07 | loss=0.3369 | loss_dist=

[2019-12-06 16:45:01,528] 
17/100 * Epoch 17 (train): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=3.7539 | _timers/batch_time=0.5335 | _timers/data_time=0.0002 | _timers/model_time=0.5332 | f1_score=6.562e-07 | loss=0.4500 | loss_dist=0.1262 | loss_mean=0.3238
17/100 * Epoch 17 (valid): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=6.5200 | _timers/batch_time=0.3120 | _timers/data_time=0.0004 | _timers/model_time=0.3115 | f1_score=6.562e-07 | loss=0.4500 | loss_dist=0.1262 | loss_mean=0.3238
[2019-12-06 16:45:08,433] 
18/100 * Epoch 18 (train): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=3.7106 | _timers/batch_time=0.5406 | _timers/data_time=0.0004 | _timers/model_time=0.5402 | f1_score=6.423e-07 | loss=0.3500 | loss_dist=0.0292 | loss_mean=0.3208
18/100 * Epoch 18 (valid): _base/lr=0.0024 | _base/momentum=0.9000 | _timers/_fps=6.4085 | _timers/batch_time=0.3172 | _timers/data_time=0.0003 | _timers/model_time=0.3169 | f1_score=6.423e-07 | loss=0.3500 | lo

[2019-12-06 16:46:47,151] 
33/100 * Epoch 33 (train): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=3.8357 | _timers/batch_time=0.5220 | _timers/data_time=0.0003 | _timers/model_time=0.5216 | f1_score=4.882e-07 | loss=0.3635 | loss_dist=0.0419 | loss_mean=0.3216
33/100 * Epoch 33 (valid): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=7.0742 | _timers/batch_time=0.2882 | _timers/data_time=0.0004 | _timers/model_time=0.2878 | f1_score=4.882e-07 | loss=0.3635 | loss_dist=0.0419 | loss_mean=0.3216
[2019-12-06 16:46:53,664] 
34/100 * Epoch 34 (train): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=3.9831 | _timers/batch_time=0.5048 | _timers/data_time=0.0003 | _timers/model_time=0.5045 | f1_score=6.530e-07 | loss=0.3549 | loss_dist=0.0352 | loss_mean=0.3196
34/100 * Epoch 34 (valid): _base/lr=0.0018 | _base/momentum=0.9000 | _timers/_fps=6.7478 | _timers/batch_time=0.3007 | _timers/data_time=0.0002 | _timers/model_time=0.3004 | f1_score=6.530e-07 | loss=0.3549 | lo

[2019-12-06 16:48:31,927] 
49/100 * Epoch 49 (train): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=3.9200 | _timers/batch_time=0.5121 | _timers/data_time=0.0003 | _timers/model_time=0.5118 | f1_score=6.040e-07 | loss=0.3543 | loss_dist=0.0413 | loss_mean=0.3130
49/100 * Epoch 49 (valid): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=6.7265 | _timers/batch_time=0.3002 | _timers/data_time=0.0003 | _timers/model_time=0.2998 | f1_score=6.040e-07 | loss=0.3543 | loss_dist=0.0413 | loss_mean=0.3130
[2019-12-06 16:48:38,501] 
50/100 * Epoch 50 (train): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=3.8640 | _timers/batch_time=0.5189 | _timers/data_time=0.0003 | _timers/model_time=0.5185 | f1_score=6.065e-07 | loss=0.4030 | loss_dist=0.0807 | loss_mean=0.3223
50/100 * Epoch 50 (valid): _base/lr=0.0013 | _base/momentum=0.9000 | _timers/_fps=7.2494 | _timers/batch_time=0.2806 | _timers/data_time=0.0003 | _timers/model_time=0.2803 | f1_score=6.065e-07 | loss=0.4030 | lo

[2019-12-06 16:50:16,824] 
65/100 * Epoch 65 (train): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=3.7952 | _timers/batch_time=0.5283 | _timers/data_time=0.0003 | _timers/model_time=0.5279 | f1_score=6.276e-07 | loss=0.3690 | loss_dist=0.0461 | loss_mean=0.3229
65/100 * Epoch 65 (valid): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=7.0192 | _timers/batch_time=0.2907 | _timers/data_time=0.0004 | _timers/model_time=0.2903 | f1_score=6.276e-07 | loss=0.3690 | loss_dist=0.0461 | loss_mean=0.3229
[2019-12-06 16:50:23,395] 
66/100 * Epoch 66 (train): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=3.8686 | _timers/batch_time=0.5184 | _timers/data_time=0.0003 | _timers/model_time=0.5181 | f1_score=6.232e-07 | loss=0.3846 | loss_dist=0.0632 | loss_mean=0.3214
66/100 * Epoch 66 (valid): _base/lr=0.0009 | _base/momentum=0.9000 | _timers/_fps=6.7778 | _timers/batch_time=0.2999 | _timers/data_time=0.0002 | _timers/model_time=0.2997 | f1_score=6.232e-07 | loss=0.3846 | lo

[2019-12-06 16:51:58,455] 
81/100 * Epoch 81 (train): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=4.1329 | _timers/batch_time=0.4868 | _timers/data_time=0.0004 | _timers/model_time=0.4863 | f1_score=5.643e-07 | loss=0.3747 | loss_dist=0.0554 | loss_mean=0.3193
81/100 * Epoch 81 (valid): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=7.6555 | _timers/batch_time=0.2649 | _timers/data_time=0.0004 | _timers/model_time=0.2645 | f1_score=5.643e-07 | loss=0.3747 | loss_dist=0.0554 | loss_mean=0.3193
[2019-12-06 16:52:04,424] 
82/100 * Epoch 82 (train): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=4.2761 | _timers/batch_time=0.4697 | _timers/data_time=0.0003 | _timers/model_time=0.4693 | f1_score=7.101e-07 | loss=0.3966 | loss_dist=0.0794 | loss_mean=0.3172
82/100 * Epoch 82 (valid): _base/lr=0.0007 | _base/momentum=0.9000 | _timers/_fps=7.4502 | _timers/batch_time=0.2736 | _timers/data_time=0.0002 | _timers/model_time=0.2734 | f1_score=7.101e-07 | loss=0.3966 | lo

[2019-12-06 16:53:41,393] 
97/100 * Epoch 97 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=3.8754 | _timers/batch_time=0.5198 | _timers/data_time=0.0002 | _timers/model_time=0.5195 | f1_score=5.737e-07 | loss=0.3501 | loss_dist=0.0321 | loss_mean=0.3180
97/100 * Epoch 97 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=6.9574 | _timers/batch_time=0.2918 | _timers/data_time=0.0003 | _timers/model_time=0.2915 | f1_score=5.737e-07 | loss=0.3501 | loss_dist=0.0321 | loss_mean=0.3180
[2019-12-06 16:53:48,017] 
98/100 * Epoch 98 (train): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=3.8625 | _timers/batch_time=0.5188 | _timers/data_time=0.0002 | _timers/model_time=0.5186 | f1_score=5.898e-07 | loss=0.3512 | loss_dist=0.0345 | loss_mean=0.3167
98/100 * Epoch 98 (valid): _base/lr=0.0005 | _base/momentum=0.9000 | _timers/_fps=6.7303 | _timers/batch_time=0.3020 | _timers/data_time=0.0004 | _timers/model_time=0.3016 | f1_score=5.898e-07 | loss=0.3512 | lo